Übung: Rolling Update 
-----------------

In dieser Übung erstellen wir mehrere Pods ab dem gleichen Image mit jeweils einem ReplicaSet, Deployment, Service und **Ingress**.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace depl

Wir Erzeugen den Pod, ReplicaSet, Deployment und Service.

Für das `Replica-Set` genügt eine Angabe `replica: 5`.

Die Pods folgen ab `spec.containers`.

Mittels `---` können mehrere Ressourcen in einer Datei zusammengefasst werden. Hier Deployment und Service.

In [ ]:
%%bash
cat <<%EOF% | kubectl --namespace depl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: bpmn-frontend
  labels:
    app: bpmn-frontend
    group: web
    tier: frontend
spec:
  ports:
  - port: 80
    targetPort: 80  
    protocol: TCP
  selector:
    app: bpmn-frontend
  type: LoadBalancer          
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: bpmn-frontend
spec:
  replicas: 5
  selector:
    matchLabels:
      app: bpmn-frontend
  template:
    metadata:
      labels:
        app: bpmn-frontend
        group: web
        tier: frontend
    spec:
      containers:
      - name: bpmn-frontend
        image: misegr/bpmn-frontend:latest
        imagePullPolicy: IfNotPresent        
        ports:
        - containerPort: 80
          name: bpmn-frontend
%EOF%

Zur Kontrolle geben wir die Erzeugten Ressourcen aus:

In [ ]:
! kubectl get pod,deployment,replicaset,service --namespace depl -o wide

Schauen wir uns zuerst die Ausgabe des laufenden Pods an, diesmal über den URL des Kubernetes API Servers und den via Ingress Ressourcen angehängten Prefix.

Der Prefix `frontend` ist fix in der Ingress Ressource hinterlegt, `index.html` ergibt sich wie der Container aufgebaut ist (Apache Server mit einer HTML Datei).

In [ ]:
! echo $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service --namespace depl bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/")/frontend

Wir wollen jedoch nicht die letzte (`latest`) Version von `bpmn-frontend` sondern die Version `V1.0`, deshalb führen wir einen Rolling Update durch, bzw. Ändern die Versionsnummer hinter dem Imagenamen.

In [ ]:
! kubectl set image deployment/bpmn-frontend bpmn-frontend=misegr/bpmn-frontend:V1.0 --namespace depl

Die Änderungen können wir uns Anzeigen lassen:

In [ ]:
! kubectl get pod,deployment,replicaset,service --namespace depl -o wide
! kubectl describe deployment/bpmn-frontend --namespace depl

Neu sollte im Titel die Versionnummer `V1.0` angezeigt werden.

In [ ]:
! echo $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service --namespace depl bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/")/frontend

RollOut
-----------

Wenn die neue Version der Software nicht wie erwartet funktoniert, können wir zur vorherigen Version zurückkehren.

Dies ist möglich, weil Kubernetes den Rollout-Verlauf von Deployment in Form von Revisionen speichert .


In [ ]:
! kubectl rollout history deployment/bpmn-frontend --namespace depl

Es sollten zwei Revisionen angezeigt werden. Die aktuelle ist die Revision 2.

Um auf die vorherige Revision zurück zu kehren, verwenden wir:

In [ ]:
! kubectl rollout undo deployment/bpmn-frontend --namespace depl

In [ ]:
! kubectl rollout history deployment/bpmn-frontend --namespace depl

Aufräumen

In [ ]:
! kubectl delete namespace depl